In [1]:
!pip install simpletransformers==0.65.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.6/312.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.9/264.9 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.3

In [2]:
import pathlib
import sklearn
from collections import Counter
from sklearn.datasets import load_files

from sklearn.metrics import classification_report
import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import matplotlib.pyplot as plt
import seaborn as sn
from collections import Counter

In [4]:
from google.colab import files
uploaded = files.upload()

Saving Google-Playstore.csv to Google-Playstore.csv


In [5]:
import io
df_google = pd.read_csv(io.BytesIO(uploaded['Google-Playstore.csv']))
df_google['index'] = df_google.index

train_google = pd.DataFrame({'name': df_google['App Name'], 'topic': df_google['Category'], 'index': df_google['index']})
print(train_google[:10])

                                                name             topic  index
0                                            Gakondo         Adventure      0
1                                Ampere Battery Info             Tools      1
2                                             Vibook      Productivity      2
3  Smart City Trichy Public Service Vehicles 17UC...     Communication      3
4                                            GROW.me             Tools      4
5                                             IMOCCI            Social      5
6                   unlimited 4G data prank free app  Libraries & Demo      6
7                              The Everyday Calendar         Lifestyle      7
8                                          WhatsOpen     Communication      8
9                   Neon 3d Iron Tech Keyboard Theme   Personalization      9


In [7]:
# keep only the labels relating to the test data
labels = ['Books & Reference', 'Sports', 'Entertainment']
filtered = train_google[train_google['topic'].isin(labels)]
filtered['topic'] = filtered['topic'].apply(lambda x: labels.index(x))
train_data = filtered[:2000]
print(train_data)

                                      name  topic  index
19          Contemporary Love Wallpaper HD      0     19
23                   Coloring Book Barbaie      2     23
27     Dj Car Sound And Sound Effects 2019      2     27
33                            Creepy Story      2     33
48             Peribahasa Bahasa Indonesia      0     48
...                                    ...    ...    ...
15439                                  TOI      2  15439
15455                        Curacao Radio      2  15455
15461              Muwatta Imam Malik Book      0  15461
15468                    Military strategy      0  15468
15471                     Ball Back System      1  15471

[2000 rows x 3 columns]


<ipython-input-7-578f73fe12cc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['topic'] = filtered['topic'].apply(lambda x: labels.index(x))


In [8]:
# change topics into integers for the model
train_data['topic'] = train_data['topic'].replace({'Books & Reference': 0, 'Entertainment': 1, 'Sports': 2})
print(train_data)

                                      name  topic  index
19          Contemporary Love Wallpaper HD      0     19
23                   Coloring Book Barbaie      2     23
27     Dj Car Sound And Sound Effects 2019      2     27
33                            Creepy Story      2     33
48             Peribahasa Bahasa Indonesia      0     48
...                                    ...    ...    ...
15439                                  TOI      2  15439
15455                        Curacao Radio      2  15455
15461              Muwatta Imam Malik Book      0  15461
15468                    Military strategy      0  15468
15471                     Ball Back System      1  15471

[2000 rows x 3 columns]


<ipython-input-8-00b052321052>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data['topic'] = train_data['topic'].replace({'Books & Reference': 0, 'Entertainment': 1, 'Sports': 2})


In [9]:
uploaded_test = files.upload()

Saving sentiment-topic-test.tsv to sentiment-topic-test.tsv


In [11]:
test_data = pd.read_csv(io.BytesIO(uploaded_test['sentiment-topic-test.tsv']),sep='\t')
test_data['index'] = test_data.index
# match topics with topics in training set
test_data['topic'] = test_data['topic'].replace({'book': 0, 'movie': 1, 'sports': 2})
print(test_data)

   sentence id                                               text sentiment  \
0            0  I wouldn't be caught dead watching the NFL if ...  negative   
1            1  Chris O'Donnell stated that while filming for ...   neutral   
2            2  The whole game was a rollercoaster ride, but L...  positive   
3            3  Zendaya slayed in Dune 2, as she does in all h...  positive   
4            4  While my favorite player was playing this matc...  negative   
5            5  My uncle's brother's neighbor's cat's veterina...   neutral   
6            6  He said that The Great Gatsby is the best nove...  negative   
7            7  I could not look away from this train wrck of ...  negative   
8            8  The film Everything Everywhere All At Once fol...   neutral   
9            9  I just finished reading pride and prejudice wh...  positive   

   topic  index  
0      2      0  
1      1      1  
2      2      2  
3      1      3  
4      2      4  
5      0      5  
6   

In [13]:
from sklearn.model_selection import train_test_split

train, dev = train_test_split(train_data, test_size=0.1, random_state=0,
                               stratify=train_data[['topic']])

In [14]:
print(len(train))
print("train:", train[['topic']].value_counts(sort=False))
train.head(3)

1800
train: topic
0        688
1        292
2        820
dtype: int64


,name,topic,index
14799,Boreal Beats,2,14799
13845,Darts Battle - 3D Dartboard Game,1,13845
14675,Fußball-Bundesliga,1,14675


In [15]:
print(len(dev))
print("dev:", dev[['topic']].value_counts(sort=False))
dev.head(3)

200
dev: topic
0        76
1        33
2        91
dtype: int64


,name,topic,index
1097,Wear Reader,0,1097
5909,Urdu Poetry Collection | Urdu SMS | Offline,0,5909
9490,"Book, The Store Boy",0,9490


In [16]:
# https://simpletransformers.ai/docs/usage/#configuring-a-simple-transformers-model
model_args = ClassificationArgs()

model_args.overwrite_output_dir=True
model_args.evaluate_during_training=True

model_args.num_train_epochs=10
model_args.train_batch_size=32
model_args.learning_rate=4e-6
model_args.max_seq_length=256

model_args.use_early_stopping=True
model_args.early_stopping_delta=0.01
model_args.early_stopping_metric='eval_loss'
model_args.early_stopping_metric_minimize=True
model_args.early_stopping_patience=2
model_args.evaluate_during_training_steps=32

In [17]:
# Checking steps per epoch
steps_per_epoch = int(np.ceil(len(train) / float(model_args.train_batch_size)))
print('Each epoch will have {:,} steps.'.format(steps_per_epoch))

Each epoch will have 57 steps.


In [18]:
model = ClassificationModel('bert', 'bert-base-cased', num_labels=3, args=model_args, use_cuda=True) # CUDA is enabled

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [19]:
print(str(model.args).replace(',', '\n')) # model args

ClassificationArgs(adafactor_beta1=None
 adafactor_clip_threshold=1.0
 adafactor_decay_rate=-0.8
 adafactor_eps=(1e-30
 0.001)
 adafactor_relative_step=True
 adafactor_scale_parameter=True
 adafactor_warmup_init=True
 adam_betas=(0.9
 0.999)
 adam_epsilon=1e-08
 best_model_dir='outputs/best_model'
 cache_dir='cache_dir/'
 config={}
 cosine_schedule_num_cycles=0.5
 custom_layer_parameters=[]
 custom_parameter_groups=[]
 dataloader_num_workers=0
 do_lower_case=False
 dynamic_quantize=False
 early_stopping_consider_epochs=False
 early_stopping_delta=0.01
 early_stopping_metric='eval_loss'
 early_stopping_metric_minimize=True
 early_stopping_patience=2
 encoding=None
 eval_batch_size=100
 evaluate_during_training=True
 evaluate_during_training_silent=True
 evaluate_during_training_steps=32
 evaluate_during_training_verbose=False
 evaluate_each_epoch=True
 fp16=True
 gradient_accumulation_steps=1
 learning_rate=4e-06
 local_rank=-1
 logging_steps=50
 loss_type=None
 loss_args={}
 manual_see

In [20]:
_, history = model.train_model(train, eval_df=dev)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:608: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/57 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1452: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1452: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/57 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1452: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


0it [00:00, ?it/s]

In [23]:
predicted, probabilities = model.predict(test_data.text.to_list())
test_data['predicted'] = predicted

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
test_data.head(5)

,sentence id,text,sentiment,topic,index,predicted
0,0,I wouldn't be caught dead watching the NFL if ...,negative,2,0,2
1,1,Chris O'Donnell stated that while filming for ...,neutral,1,1,0
2,2,"The whole game was a rollercoaster ride, but L...",positive,2,2,2
3,3,"Zendaya slayed in Dune 2, as she does in all h...",positive,1,3,2
4,4,While my favorite player was playing this matc...,negative,2,4,2


In [25]:
print(classification_report(test_data['topic'], test_data['predicted']))

              precision    recall  f1-score   support

           0       0.50      1.00      0.67         3
           1       0.00      0.00      0.00         4
           2       0.75      1.00      0.86         3

    accuracy                           0.60        10
   macro avg       0.42      0.67      0.51        10
weighted avg       0.38      0.60      0.46        10



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
